In [1]:
import sys
import json
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.auto import tqdm
from tqdm.keras import TqdmCallback
sys.path.append("../../../../")
from src.Evaluation import plot_model_history
from src.ModelBuilder import get_MLP, get_MCDCNN, get_Time_CNN, get_FCN, get_Encoder
from src.LoadData import get_all_datasets_test_train_np_arrays
from src.Helpers import append_to_csv

2023-01-18 18:43:06.004415: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 18:43:06.131923: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-18 18:43:07.007008: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bwhpc/common/devel/cuda/11.8/lib64
2023-01-18 18:43:07.007082: W tensorflow/compil

In [2]:
path_to_datasets = "../../../../datasets"

In [3]:
datasets_test_train_data = get_all_datasets_test_train_np_arrays(path_to_datasets)

In [4]:
models_getter = [get_Encoder, ]
models_names = ["Encoder", ]

In [5]:
# tf.debugging.set_log_device_placement(True)
devices = tf.config.list_logical_devices('GPU') # Use this to run training just on GPUs
#devices = tf.config.list_logical_devices()
strategy = tf.distribute.MirroredStrategy(devices)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2023-01-18 18:43:16.422707: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 18:43:17.618240: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 88 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:89:00.0, compute capability: 7.0
2023-01-18 18:43:17.619080: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 88 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:8a:00.0, compute capability: 7.0
2023-01-18 18:43:17.639586: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:735] failed to allocate 88.25M (92

In [6]:
devices

[LogicalDevice(name='/device:GPU:0', device_type='GPU'),
 LogicalDevice(name='/device:GPU:1', device_type='GPU')]

In [7]:
path_persist_results = "./training_res.csv"
if not (csv_path := Path(path_persist_results)).exists():
    csv_path.touch()
if len(csv_path.read_text()) == 0:
    # Only write if the file is empty
    columns = ["dataset_name", "model_name", "test_loss", "test_acc", "confusion_matrix", "history"]
    append_to_csv(path_persist_results, columns)

In [8]:
epochs = 30
batch_size = 20
validation_split = 0.0

In [9]:
with strategy.scope():
    for ds_name, ds_data in tqdm(datasets_test_train_data.items(), unit='dataset'):
        print("Dataset name: ", ds_name)
        x_test, y_test = ds_data["test_data"]
        x_train, y_train = ds_data["train_data"]

        input_size = x_train.shape[1]
        output_size = len(np.unique(y_train))

        for get_model, model_name in tqdm(list(zip(models_getter, models_names)), unit='model', desc=f'Train on "{ds_name}"'):
            print("Model name: ", model_name)
            model = get_model(input_size, output_size)
            model.compile(optimizer='SGD',
                          loss='sparse_categorical_crossentropy',
                          metrics=['accuracy'])
            history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=validation_split, callbacks=[TqdmCallback(verbose=0, desc=model_name)], verbose=0)
            test_loss, test_acc = model.evaluate(x_test, y_test)
            # TODO: add confusion matrix

            row = [ds_name,
                  model_name,
                  test_loss,
                  test_acc,
                  None, # TODO: persist confusion matrix to further analysis
                  json.dumps(history.history)]
            append_to_csv(path_persist_results, row)

  0%|          | 0/11 [00:00<?, ?dataset/s]

Dataset name:  abnormal_heartbeat


Train on "abnormal_heartbeat":   0%|          | 0/1 [00:00<?, ?model/s]

Model name:  Encoder
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


Encoder: 0epoch [00:00, ?epoch/s]

INFO:tensorflow:batch_all_reduce: 21 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 21 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/devic

2023-01-18 18:43:34.109302: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 90.48MiB (rounded to 94873600)requested by op model/conv1d/Conv1D
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-01-18 18:43:34.109332: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2023-01-18 18:43:34.109341: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 47, Chunks in use: 47. 11.8KiB allocated for chunks. 11.8KiB in use in bin. 426B client-requested in use in bin.
2023-01-18 18:43:34.109363: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 9, Chunks in use: 9. 4.5KiB allocated for chunks. 4.5KiB in use in bin. 4.5KiB client-requested in use in bin.
2023-01-18 18:43:34.109368: I tensorflow/tsl/frame

ResourceExhaustedError: Graph execution error:

Detected at node 'model/conv1d/Conv1D' defined at (most recent call last):
    File "/usr/lib64/python3.8/threading.py", line 890, in _bootstrap
      self._bootstrap_inner()
    File "/usr/lib64/python3.8/threading.py", line 932, in _bootstrap_inner
      self.run()
    File "/opt/bwhpc/common/jupyter/tensorflow/2023-01-02/lib/python3.8/site-packages/keras/engine/training.py", line 1222, in run_step
      outputs = model.train_step(data)
    File "/opt/bwhpc/common/jupyter/tensorflow/2023-01-02/lib/python3.8/site-packages/keras/engine/training.py", line 1023, in train_step
      y_pred = self(x, training=True)
    File "/opt/bwhpc/common/jupyter/tensorflow/2023-01-02/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 65, in error_handler
      return fn(*args, **kwargs)
    File "/opt/bwhpc/common/jupyter/tensorflow/2023-01-02/lib/python3.8/site-packages/keras/engine/training.py", line 561, in __call__
      return super()

# Example load training results and display evaluation results

In [ ]:
df_training_res = pd.read_csv(path_persist_results)

In [ ]:
df_training_res

In [ ]:
history = df_training_res['history'][0]

In [ ]:
plot_model_history(json.loads(history), epochs=epochs)